In [1]:
import numpy as np

space = [-1, 1]
runs = 1000
max_iterations = 1000  #max

def misclassified(a, b):
    return np.where(a != b)[0] #return misclassified points

def error_val(w, test_data, N):
    X = test_data[:N, :-1]
    y = test_data[:, -1]

    # add synthetic column
    ones = np.ones((X.shape[0], 1))
    X = np.concatenate((ones, X), axis=1)

    y_estimates = np.sign(np.dot(X, w.T))
    #calculate average error rate
    return misclassified(y, y_estimates).shape[0] / N


def PLA(data, N):
  x = data[:, :-1] #training data
  y = data[:N:, -1]

  ones = np.ones((N, 1))
  x = np.concatenate((ones, x), axis=1)

  w = np.zeros(x.shape[1]) #initialize weight vector with 0s

  count = 0

  while count < max_iterations:
    predictions = np.sign(np.dot(x, w.T))
    missed = misclassified(y, predictions).tolist()
    if not missed: #break if no misclassifications
      break

    rand_idx = np.random.choice(missed)
    w = w + y[rand_idx] * x[rand_idx] #update weight vec

    count += 1

  return w, count

def generate_target_function():
    #randomly pick two points to define target function
    p1 = np.random.uniform(*space, 2)
    p2 = np.random.uniform(*space, 2)

    #define slope and intercept for the line
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0])
    y_intercept = p1[1] - slope * p1[0]

    #return the target function
    return lambda x: slope * x + y_intercept

def prepare_data(N, target_function):
    def target(point):
        return target_function(point[0])

    X = []
    Y = []
    for i in range(N):
        new_pt = np.random.uniform(*space, 2)  #generate random points
        y = target(new_pt)
        difference = y - new_pt[1]

        X.append(new_pt)
        Y.append(difference)

    targets = np.sign(Y)  #convert differences to -1 or 1
    data = np.concatenate((X, targets.reshape(-1, 1)), axis=1)
    return data


def run(N, target_function):
    #generate training and test data using same target function
    train_data = prepare_data(N, target_function)
    test_data = prepare_data(N, target_function)

    #train PLA on the training data
    w, iterations = PLA(train_data, N)

    #calculate error on the test data
    error = error_val(w, test_data, N)

    return iterations, error

def main():
    for N in [10, 100]:
        iterations = []
        errors = []

        #generate a target function
        target_function = generate_target_function()

        for _ in range(runs):
            c, e = run(N, target_function)
            iterations.append(c)
            errors.append(e)

        print(f'Avg iterations for N={N} = {np.mean(iterations):.3f}')
        print(f'Avg error for N={N} = {np.mean(errors):.3f}')
        print('')

if __name__ == "__main__":
    main()

Avg iterations for N=10 = 6.924
Avg error for N=10 = 0.073

Avg iterations for N=100 = 80.026
Avg error for N=100 = 0.014



In [ ]:
#the 8 target functions
target_functions = [[0, 0, 0], [0, 0, 1], [0, 1, 0],  [0, 1, 1],
                    [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]

#the four hypothesises
hypothesis = [[1, 1, 1], [0, 0, 0], [0, 0, 1], [1, 1, 0]]

scores = []

for hypo in hypothesis:
  one_point = 0
  two_point = 0
  three_point = 0
  for target in target_functions:
    curr = 0
    if target[0] == hypo[0]:
      curr += 1
    if target[1] == hypo[1]:
      curr += 1
    if target[2] == hypo[2]:
      curr += 1

    if (curr >= 3):
      three_point += 1
    elif (curr >= 2):
      two_point += 1
    elif (curr >= 1):
      one_point += 1

  #calculate the score for each hypothesis
  score = 3 * three_point + 2 * two_point + 1 * one_point
  print(f'hypo: {hypo} = score: {score}')

hypo: [1, 1, 1] = score: 12
hypo: [0, 0, 0] = score: 12
hypo: [0, 0, 1] = score: 12
hypo: [1, 1, 0] = score: 12
